In [37]:
import re
import numpy as np
import pandas as pd
import json

matrix = "matrix20170512.xlsx"
path = "/Users/vyacheslavtykhonov/projects/parthenos/%s" % matrix
path = "/Users/vyacheslavtykhonov/projects/parthenos-widget/data/matrix20170515.xlsx"
xl = pd.ExcelFile(path)
xl.sheet_names[1]


u'ARCHAEOLOGY'

In [38]:
def contents(path, tabname):
    df = pd.read_excel(path, sheetname=tabname, header=None, skiprows=0)
    data = df #.transpose()
    content = {}
    cindex = {}
    for index in data.index:
        row = data.ix[index]
        if str(row[1]) != 'nan':
            content[row[0]] = row[1]
            cindex[row[1]] = row[1]
        else:
            content[row[0]] = ''
            cindex[row[0]] = row[0]
    data = {}
    mindex = []
    for x in cindex:
        mindex.append(x)
    data['index'] = mindex
    data['contents'] = content
    return (df, data)

(df, dataset) = contents(path, xl.sheet_names[0])
data = json.dumps(dataset, ensure_ascii=False, sort_keys=True, indent=4)
print df.columns
def read_topics(path, tabname):
    forbidden = ['POLICY', 'POLICY LINK', 'COUNTRY', 'ORGANISATION', 'ORGANISATION LINK', 'PROJECT']
    df = pd.read_excel(path, sheetname=tabname, skiprows=1)
    columns = []
    for colname in df.columns:
        if colname not in forbidden:
            columns.append(colname)
    return columns

df = read_topics(path, xl.sheet_names[3])
df

Int64Index([0, 1], dtype='int64')


[u'BEST PRACTICE',
 u'FINDABLE \n(persistent identifier; rich and indexed metadata, citation) ',
 u'ACCESSIBLE \n(data and metadata are retrievable via an exchange protocol; protocol is open and free) ',
 u'INTEROPERABLE \n(use of (meta)data standards, use of vocabularies, reference to other metadata)',
 u'REUSABLE \n(provenance; usage licence; standards)',
 u'FIELD (among Language Studies)',
 u'SUGGESTED METADATA FORMAT',
 u'PREFERRED DATA FORMAT',
 u'LEGAL FRAMEWORK',
 u'OPEN ACCESS POLICIES',
 u'PRIVACY AND SENSITIVE DATA',
 u'DATA CREATION',
 u'METADATA CREATION',
 u'NAMING FILE CONVENTION',
 u'CITATION GUIDELINES',
 u'RESEARCH ETHICS',
 u'ACCESSIBILITY',
 u'LONG TERM PRESERVATION',
 u'MANTAIN THE QUALITY AND INTEGRITY OF DATA AND DATASETS',
 u'TRUSTED DIGITAL REPOSITORY',
 u'ARCHIVAL STORAGE',
 u'DATA MANAGEMENT ',
 u'PRESERVATION PLANNING',
 u'Data Sharing Protocols',
 u'ASSESSEMNT OF REPOSITORIES',
 u'UNIQUE AND PERSISTENT IDENTIFIERS',
 u'SELECTION OF RESEARCH DATA',
 u'THESAUR

In [39]:
majorkey = "POLICY"
location = "COUNTRY"
df = pd.read_excel(path, sheetname="ARCHAEOLOGY", header = 1)
d = df[df[majorkey] == 'ADS - Archaeology Data Service']
data = d.transpose()
df = df[pd.notnull(df[majorkey])]
#df.index = df[majorkey]
major = {}
for polname in df[majorkey]:
    polline = df[df[majorkey] == polname][location]
    print polline.iloc[0]
#    major[polname] = str(polline[location])
#    print df[df[majorkey] == polname][location]
#major

United States
United States
Europe
United Kingdom
Romania
Europe
Europe
Europe
Europe
nan
Austria
United States
Netherlands
Germany
Germany
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Germany, Europe
Netherlands
Europe
Europe
Europe
Germany
Unitede States
France
Hungary
Netherlands
United Kingdom
United States
Canada


In [40]:
#df.ix[1][0]
#for x in df.index:
#    print x

In [41]:
xl = pd.ExcelFile(path)

xl.sheet_names

[u'CONTENTS',
 u'ARCHAEOLOGY',
 u'DIGITAL REPOSITORY',
 u'LANGUAGE STUDIES',
 u'SOCIAL SCIENCE',
 u'HISTORY',
 u'CULTURAL HERITAGE INSTITUTION']

In [42]:
def fair(tabname):
    df = pd.read_excel(path, sheetname=tabname, header = 1)
    fairfilter = {}
    for column in df.columns:
        if re.search("FINDABLE", column, re.DOTALL): #, re.MULTILINE):
            fairfilter['FINDABLE'] = column
        if re.search("ACCESSIBLE", column, re.DOTALL): #, re.MULTILINE):
            fairfilter['ACCESSIBLE'] = column
        if re.search("INTEROPERABLE", column, re.DOTALL): #, re.MULTILINE):
            fairfilter['INTEROPERABLE'] = column
        if re.search("REUSABLE", column, re.DOTALL): #, re.MULTILINE):
            fairfilter['REUSABLE'] = column
    return (df, fairfilter)

def fairfilter(df, command):
    fairmatrix = {}
    fairmatrix['f'] = df[filters['FINDABLE']]=='X'
    fairmatrix['a'] = df[filters['ACCESSIBLE']]=='X'
    fairmatrix['i'] = df[filters['INTEROPERABLE']]=='X'
    fairmatrix['r'] = df[filters['REUSABLE']]=='X'

    resultmatrix = pd.DataFrame()
    for acr in command:
        if resultmatrix.empty:   
            resultmatrix = fairmatrix[acr]
        else:
            resultmatrix = resultmatrix & fairmatrix[acr]
            
    return df[resultmatrix]
        
sheetname="ARCHAEOLOGY"
(df, filters) = fair(sheetname)
x = fairfilter(df, 'fr')

f = df[filters['FINDABLE']]=='X'
a = df[filters['ACCESSIBLE']]=='X'
i = df[filters['INTEROPERABLE']]=='X'
r = df[filters['REUSABLE']]=='X'
#x = f & r
#x = df[f & a & i & r]
#x.transpose()
#x = f & a & i & r
#for thiscol in x.columns:
#    print thiscol + "\n"
x= df[f & a]
x

,POLICY,POLICY LINK,COUNTRY,ORGANISATION,ORGANISATION LINK,PROJECT,"FINDABLE Metadata, Documentation, Identifiers for Citation",ACCESSIBLE (data and metadata are retrievable via an exchange protocol; protocol is open and free) - Licence,"INTEROPERABLE (use of (meta)data standards, use of vocabularies, reference to other metadata)",REUSABLE (provenance; usage licence; standards),...,TRUSTED DIGITAL REPOSITORY,ARCHIVAL STORAGE,DATA MANAGEMENT,PRESERVATION PLANNING,ASSESSMENT OF REPOSITORY,SUSTAINABILITY AND FUNDING,UNIQUE AND PERSISTENT IDENTIFIER,SELECTION OF RESEARCH DATA,"THESAURI, GLOSSARIES, AUTHORITY FILES",LINKED OPEN DATA
3,NaN,NaN,United Kingdom,ADS - Archaeology Data Service,http://archaeologydataservice.ac.uk/,ARIADNE,X,X,NaN,X,...,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,NaN,NaN,Italy,MIBACT-ICCD (Central Institute for Cataloguing...,http://www.internetculturale.it/opencms/opencm...,"CULTURAITALIA, EUROPEANA, SITAR, ATHENA (finis...",X,X,X,X,...,NaN,NaN,NaN,NaN,NaN,NaN,X,X,X,X


In [49]:
matrix = []
matrix.append(df[filters['FINDABLE']]=='X')
#matrix.append(df[filters['ACCESSIBLE']]=='X')
#matrix.append(df[filters['REUSABLE']]=='X')
fmatrix = matrix[0]
for i in range(1, len(matrix)):
    print i
    fmatrix = (fmatrix) & (matrix[i])
#x = df[matrix[0] & matrix[1]]
y = df[fmatrix]


In [84]:
selection = ['POLICY','COUNTRY','POLICY LINK']
#selected = [filters['FINDABLE']]
#selection.append(filters['FINDABLE'])
selected = ["Aerial Documentation"]
selection.append("Aerial Documentation")
ymatrix = y[selection]
result = {}
if selected:
    for col in selected:
        print col
        rowarray = []
        for row in ymatrix.index:
            rowsubset = ymatrix.ix[row]
            rowresult = {}
            for col in selected:
                if rowsubset[col] == 'X':
                    d = rowsubset.to_dict() #orient='records')
                    rowarray.append(d)
        result[col] = rowarray
result

Aerial Documentation


{'Aerial Documentation': [{u'Aerial Documentation': u'X',
   u'COUNTRY': u'Germany',
   u'POLICY': nan,
   u'POLICY LINK': nan},
  {u'Aerial Documentation': u'X',
   u'COUNTRY': u'Ireland',
   u'POLICY': nan,
   u'POLICY LINK': nan}]}

In [44]:
def frametokeys(df):
    allkeys = {}
    for item in df.columns:
        allkeys[item] = ''
    return allkeys

In [45]:
def frametovalues(df):
    allvalues = {}
    skip = 0
    for thisindex in df.index:
        line = df.ix[thisindex]
        for col in line:
            try:
                allvalues[str(col)] = ''
            except:
                skip = 1
                
        finalvalues = []
        for item in sorted(allvalues):
            finalvalues.append(item)
    return finalvalues

In [46]:
allkeys = frametokeys(df)
allkeys

{u'3D documentation': '',
 u'ACCESSIBILITY': '',
 u'ACCESSIBLE \n(data and metadata are retrievable via an exchange protocol; protocol is open and free) - Licence ': '',
 u'ARCHIVAL STORAGE': '',
 u'ASSESSMENT OF REPOSITORY': '',
 u'Aerial Documentation': '',
 u'COUNTRY': '',
 u'DATA CREATION': '',
 u'DATA MANAGEMENT': '',
 u'Dendrochronology standards': '',
 u'FINDABLE \nMetadata, Documentation, Identifiers for Citation': '',
 u'Field Documentation': '',
 u'INTEROPERABLE \n(use of (meta)data standards, use of vocabularies, reference to other metadata) ': '',
 u'LEGAL FRAMEWORK': '',
 u'LINKED OPEN DATA': '',
 u'LONG TERM PRESERVATION': '',
 u'Lidar - GPS - Spatial Documentation': '',
 u'MANTAIN THE INTEGRITY OF DATA AND DATASETS': '',
 u'METADATA CREATION': '',
 u'NAMING FILE CONVENTION': '',
 u'ORGANISATION': '',
 u'ORGANISATION LINK': '',
 u'POLICY': '',
 u'POLICY LINK': '',
 u'PREFERRED DATA FORMATS': '',
 u'PRESERVATION PLANNING': '',
 u'PRIVACY, SENSITIVE DATA ': '',
 u'PROJECT':

In [47]:
allvalues = frametovalues(df)
allvalues

['\nhttps://dans.knaw.nl/en\n',
 ' ',
 ' The Alexandria Archive Institute',
 '3D ICONS',
 '3D ICONS; ArcheoLandscapes',
 '3D Icons Guidelines',
 'A Framework for Transforming Archaeological Databases to Linked Ontological Datasets. In Computer Applications and Quantitative Methods in Archaeology',
 'A standard and guide to best practice for archeological archiving in Europe',
 'ADS - Archaeology Data Service',
 'AHEROVEST',
 'ARCHES',
 'ARCHES; EAC',
 'ARHEO - Arheovest Association Timisoara',
 'ARIADNE',
 'ARIADNE, IANUS, DARIAH.DE; Archeolandscapes',
 'ARIADNE; 3D Icons',
 'ARIADNE; ArcheoLandscape; ',
 'ARKAS archaeological data standardisation guidelines',
 'ARUP-CAS Institute of Archaeology of the Academy of Sciences',
 'ATHENA ',
 'Archaeological Documentation',
 'Archaeology Data Service/ Digital Antiquity. Guide to Good Practice. ',
 'Art & Architecture Thesaurus - AAT ',
 'Austria',
 'BUNDESDENKTMALAMT',
 'Book Chapter with examples from the ADS archological data Service',
 'B